In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Preface

I'm new to Kaggle, and markdown. 

I'm also following notebook https://www.kaggle.com/josegabrielgonzalez/exploratory-data-analysis-on-pokemon-s-dataset to see what I can learn.

# 1. Basic Information

In [ ]:
data = pd.read_csv('../input/pokemon/Pokemon.csv', index_col='#')

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.describe()

In [ ]:
data.info()

# 1.1 Basic Information on Variables

In [ ]:
len(data.Name.unique())

All Names are unique 

In [ ]:
pokemon_types = list(data['Type 1'].unique())
number_of_pokemon_types = len(pokemon_types)
print('These are the pokemon types: {}. \nThere are {} of them.'.format(', '.join(sorted(pokemon_types)), number_of_pokemon_types))

In [ ]:
second_types_not_in_type_1 = set(data['Type 2'].unique()) - set(pokemon_types)
print('Following Types are not found in Type 1:', second_types_not_in_type_1)

So Type 2 are the same as Type 1, except some pokemon doesn't have a second type. 

In [ ]:
stats = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
stat_describe = pd.DataFrame({stat: data[stat].describe() for stat in stats})
stat_describe

All pokemon have atleast 1 HP.

Max Value for base stats in peaks at 255.

The range of values seems correct. 

In [ ]:
generation_counts = data.Generation.value_counts()
generation_counts

6 Generations.

The latest generation 6 has the least by a significant margin. The data may not be the latest, or new pokemon has not come out. 

In [ ]:
data.Legendary.describe()

In [ ]:
data.Legendary.value_counts()

So a pokemon is either legendary or not.

Most pokemon are not legendary.

# 2. Data Sanitization

In [ ]:
# df.index = df.index.str.replace(".*(?=Mega)", "") # Eliminating the Mega prefix
data.Name.replace(".*(?=Mega)", "", regex=True, inplace=True)
data.head()

Removing the name before the Mega

In [ ]:
data.Name[data.Name.str.contains('Mega ')]

Looks like there aren't any non-Mega pokemon with "Mega " in it's name.

In [ ]:
data['Mega'] = data.Name.str.contains('Mega ')

In [ ]:
data.head()

Marking Megas as they should be treated separately from a regular pokemon. 

In [ ]:
reg_df = data[data.Mega == False]
reg_df.head()

In [ ]:
mega_df = data[data.Mega]
mega_df.head()

Separating Megas from Non-Mega Pokemon

In [ ]:
common_df = reg_df[reg_df.Legendary == False]
common_df.tail()

# 3. Analysis

In [ ]:
plt.figure(figsize=(16, 6))
sns.histplot(data=common_df, x='Total', hue='Generation', element='step', palette='Set2')

In [ ]:
g = sns.FacetGrid(data=common_df, col='Generation', col_wrap=3, aspect=2)
g.map(sns.histplot, 'Total')

Gen 3 is the only generation with pokemon > 600 total stats. What's going on there?

Gen 6 has a smaller count, because it has less pokemon.

In [ ]:
common_df[(common_df.Generation == 3) & (common_df.Total > 600)]

Slacking is the exception. A non-legendary, non-mega pokemon with stats over 600. 